In [1]:
using Plots, Statistics, CSV, DataFrames, Printf

# Test of a Random Number Generator Using Random Walks
### Gabe Schumm
### Due September 29th, 2021

## Random Number Geneator and Associated Functions

In [2]:
function rand_int(r)
    if typeof(r) != UInt64
        r = convert(UInt64, r)
    end
    
    a::UInt64 = 2862933555777941757
    c::UInt64 = 1013904243
    
    return (a*r) + c
end

function get_bit(r, i::Int64)
    if typeof(r) != UInt64
        r = convert(UInt64, r)
    end
    return convert(Int64, (r >>> (i-1)) &1)
end


#Stirling's approximation for log(y!)
#Only used when y > 20
function log_fact_appox(y::Int64)
    return y *log(y) - y + log(y*(1+(4*y)*(1+2*y)))/6 + log(pi)/2
end


function P_n(x::Int64, n::Int64)
    
    log_n_fact =  log_fact_appox(n) #n>20 for this assignment
    
    np = (n+x) ÷ 2
    nm = (n-x) ÷ 2
    
    if np > 20
        log_np_fact = log_fact_appox(np)
    else
        log_np_fact = log(factorial(np))
    end
    
    if nm > 20
        log_nm_fact = log_fact_appox(nm)
    else
        log_nm_fact = log(factorial(nm))
    end
    
    log_P = log_n_fact - (n * log(2)) - log_np_fact -log_nm_fact
    
    return exp(log_P)
end


P_n (generic function with 1 method)

In [3]:
function random_walk(N_w::Int64, n::Int64, r_0::UInt64)
    #N_w is number of random walk simulations to run
    #n is number of steps in each random walk simulation
    #r_0 is random UInt64 seed
    
    counts_array = zeros(64, 2*n + 1) #histogram of final walk distance for each bit
                                      # one row for each bit, 2*n + 1 columns for each
                                      #final distance value (odd and even)
    r = r_0
    for w =1:N_w
        walk = zeros(64) #random walk distance for each of 64 bits

        for i=1:n
            r = rand_int(r)
            walk += 2 .* get_bit.(r, 1:64) .- 1 #step added for each bit 
        end

        for b=1:64
            x_final = convert(Int64,  walk[b] + n + 1) #final distance converted to index of counts_array
            counts_array[b, x_final] +=1 #final distance count incremented for each bit                                         
        end
    end
    
    delta = zeros(64, 2*n + 1) #array for delta value of each bit and final walk distance
    
    for b=1:64
        delta[b, :] = (counts_array[b, :]/N_w) .- P_n.(-n:n, n) #for each bit, calculate delta
                                                                #based on counts array
    end
    
    delta2 = sum(delta[:, 1:2:end] .^2, dims=2) #sum delta^2 along all walk lengths, for each bit
                                                #only include even x values, assuming n is even
    return counts_array, sqrt.(delta2) 
    
end


random_walk (generic function with 1 method)

In [4]:
function run_random_walk(input)
    n_params = size(input)[1] #number of N_w values in input
    
    N_w_delta = zeros(64, n_params) #sqrt(N_w) * delta_b vs. bit number for each simulation
    
    #DataFrames containing results of each simulation, for each bit (64 df's)
    #x = final position, only even x values, assuming n is even
    #P_exact = probability of final position from binomial dist., takes x value and n value
    dist_dfs = [DataFrame(x = -100:2:100, P_exact = P_n.(-100:2:100, input[1, "n"], )) for i =1:64]
    
    for i=1:n_params
        N_w = input[i, "N_w"] #number of random walk simulations to run
        
        counts_array, delta = random_walk(N_w, input[i, "n"], input[i, "r_0"]) #run simulation with given paramters
        
        N_w_delta[:, i] = delta * sqrt(N_w) #calculate sqrt(N_w) * delta_b and store value
        
        for b=1:64
            #column for probability of final position for given number of simulations (N_w)
            dist_dfs[b][!, string("P_", N_w)] = counts_array[b,:][1:2:end] / N_w 
        end
    end
        
     return dist_dfs, N_w_delta
end

run_random_walk (generic function with 1 method)

### Simulation Parameters 

In [50]:
n_values = 100 * ones(Int64, 6) #n=100 for all 6 runs
N_w_values = 10 .^ (2:7) #Number of random walk simulations run, 10^2 - 10^7
r_0_values = rand(UInt64, 6) #random seed for each run

input = DataFrame(n = n_values, N_w = N_w_values, r_0 = r_0_values); #organize inputs in DataFrame

### Run Simulation

In [63]:
#first(dist_dfs[1], 5)
dist_dfs[64][47:55,:]

,x,P_exact,P_100,P_1000,P_10000,P_100000,P_1000000,P_10000000
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-8,0.0579584,0.06,0.059,0.0612,0.05851,0.058397,0.0578962
2,-6,0.0665905,0.07,0.053,0.0655,0.06628,0.066763,0.0666624
3,-4,0.073527,0.08,0.083,0.0724,0.07307,0.073371,0.0735431
4,-2,0.0780287,0.03,0.082,0.0795,0.07705,0.078073,0.0781397
5,0,0.0795892,0.08,0.079,0.0762,0.0792,0.079293,0.0797125
6,2,0.0780287,0.07,0.082,0.0756,0.07842,0.078475,0.0781047
7,4,0.073527,0.04,0.089,0.0757,0.07448,0.073433,0.0735032
8,6,0.0665905,0.03,0.065,0.0672,0.0674,0.066432,0.0665313
9,8,0.0579584,0.04,0.05,0.0588,0.05725,0.057926,0.0578475


In [65]:
d = DataFrame(N_w_delta, :auto)
d = rename(df, Dict([names(df)[i] => string(N_w_values[i]) for i=1:6]))
first(d, 5)

,100,1000,10000,100000,1000000,10000000
,Float64,Float64,Float64,Float64,Float64,Float64
1,9.47191,29.9528,94.7191,299.528,947.191,2995.28
2,9.47191,29.9528,94.7191,299.528,947.191,2995.28
3,6.32686,20.231,94.7191,299.528,632.686,2995.28
4,3.93437,20.0073,63.2686,200.073,523.193,2000.73
5,4.63834,10.9778,52.3193,165.448,462.04,1654.48


### Export Simulation results

In [66]:
CSV.write("d.csv", d);    

In [11]:
for b=0:63
    filename = @sprintf("p%02i.csv",b)
    CSV.write(string("bit_prob_dists/", filename), dist_dfs[b+1])
end

### Exploring Periodicity of Each Bit

In [72]:
r = 502
sequence = []

b = 5 #bit of interest
p = 2^b #periodicity of bit of interest

for i=1:100
    r = rand_int(r)
    push!(sequence, get_bit(r, b))
end

In [74]:
p=2^b
println(sequence[1:p] == sequence[1+p:2*p]) #first sequence of p bits = next sequence of p bits 

println(sum(2 .* sequence[1:p] .- 1)) #sum of cycle of steps = 0

true
0


In [75]:
r=502
for i =1:15
    #print iteration number, first three bits of string, r mod 4, and r mod 8
    println(i-1, " & ",  string(bitstring(r)[62:end]), " & ", r % 4, " & ", r % 8)
    r = rand_int(r)
end

0 & 110 & 2 & 6
1 & 001 & 1 & 1
2 & 000 & 0 & 0
3 & 011 & 3 & 3
4 & 010 & 2 & 2
5 & 101 & 1 & 5
6 & 100 & 0 & 4
7 & 111 & 3 & 7
8 & 110 & 2 & 6
9 & 001 & 1 & 1
10 & 000 & 0 & 0
11 & 011 & 3 & 3
12 & 010 & 2 & 2
13 & 101 & 1 & 5
14 & 100 & 0 & 4
